In [1]:
import pandas as pd
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from keras import layers
from keras.models import Sequential

EPOCHS= 30
act= "swish"
opt= tf.keras.optimizers.Adam(learning_rate=0.001)
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import FastText
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, Dropout, Flatten, Bidirectional, Conv1D, concatenate, SpatialDropout1D
from tensorflow.keras.layers import MaxPooling1D, MaxPooling2D, GlobalMaxPooling1D, GlobalAveragePooling2D, GlobalAveragePooling1D 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping

In [2]:
path = "/content/drive/MyDrive/Datasets/g10words-15authors.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
Y_train = df['USER']
Y_train.head()

0    AnaghaAcharya
1    AnaghaAcharya
2    AnaghaAcharya
3    AnaghaAcharya
4    AnaghaAcharya
Name: USER, dtype: object

In [5]:
df['label'] = le.fit_transform(Y_train)

In [6]:
df[['USER', 'label']].drop_duplicates(keep='first')

USER  label
0        AnaghaAcharya      1
1060       AUThackeray      0
2266         bb_thorat      6
3576      BodkheShilpa      2
4964        kolhe_amol      7
7658      mulshikar712      8
8344       PawarSpeaks      3
10825     rajeshtope11      9
13138        RRPSpeaks      4
15959      samant_uday     10
17738    sangrambhaiya     11
19479    Subhash_Desai      5
21148  uddhavthackeray     12
22726    vijayshivtare     13
23782  wankhedeprafull     14

In [7]:
X_train = df['clean_text']
Y_train = df['label']

In [8]:
sentences_ted = []

X_train = X_train.astype(str)

for sent_str in X_train:
  tokens = sent_str.lower().split()
  sentences_ted.append(tokens)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size

80969

In [9]:
lens_train = [len(i) for i in X_train]
lens_test = [len(i) for i in X_test]
lens = lens_train + lens_test

maxlen = np.max(lens)
print('Max len:', maxlen)

Max len: 55


In [10]:
s = pd.Series(lens)
s.value_counts()

34    1070
36    1030
33    1023
37     981
35     975
32     936
15     885
12     872
14     844
13     839
31     830
11     819
30     815
29     808
16     779
38     767
28     733
25     727
27     693
23     693
22     687
17     681
18     668
24     665
19     658
39     652
26     652
20     613
21     589
40     586
41     560
43     401
42     317
44     176
45     120
46      82
47      59
48      39
49      23
50      23
51      15
52       4
10       3
53       2
9        1
55       1
dtype: int64

In [11]:
X_train = pad_sequences(X_train, padding='post', maxlen=47)
X_test = pad_sequences(X_test, padding='post', maxlen=47)

In [12]:
model_ted = FastText(sentences_ted, size=300, window=5, min_count=5, workers=8,sg=1)

In [13]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, 300))
for word,i in word_index.items():
    try:
        embedding_vector = model_ted.wv[word]
    except:
        print(word, 'not found')
        embedding_vector = None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

91 not found
०१ not found
०९ not found
०८ not found
ह not found
ढीग not found
94 not found
34 not found
62 not found
॰ not found
४३ not found
गय not found
अढळ not found
66 not found
९२ not found
४९ not found
सढळ not found
84084 not found
ज not found
र not found
५८ not found
81 not found
71 not found
85 not found
09 not found
८४ not found
०७ not found
४६ not found
भ not found
” not found
छळ not found
2a not found
38 not found
01 not found
उ not found
े not found
० not found
49 not found
74 not found
जज not found
थॉमस not found
आए। not found
ऑइल not found
32 not found
07 not found
92 not found
76 not found
८८ not found
9249 not found
9496 not found
ओक not found
मठ not found
बढो not found
ईसुफ not found
०४ not found
‘ब’ not found
ऊठ not found
'split not found
'ए not found
मढ not found
42 not found
64 not found
67 not found
बूथ not found
यह not found
ऐ not found
लुई not found
इद not found
ी not found
ओठ not found
'ण' not found
hr not found
छ not found
ऊब not found
68 not found
82 not found

In [14]:
myCNN = Sequential()
myCNN.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=47))
myCNN.add(Conv1D(filters = 32 , kernel_size=3,activation='relu'))
myCNN.add(GlobalMaxPooling1D())
myCNN.add(Dropout(0.3))
myCNN.add(Dense(15, activation='softmax'))
myCNN.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])

myCNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 300)           24290700  
                                                                 
 conv1d (Conv1D)             (None, 45, 32)            28832     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 15)                495       
                                                                 
Total params: 24,320,027
Trainable params: 24,320,027
Non-trainable params: 0
____________________________________________

In [15]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
ec = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
reduce_lr_callback = ReduceLROnPlateau( monitor='val_acc', 
                                       factor=0.1, 
                                       patience=2, 
                                       min_lr=0.0005, 
                                       verbose=2)

In [16]:
history = myCNN.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 
loss, accuracy = myCNN.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myCNN.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
159/159 [==============================] - 19s 40ms/step - loss: 2.2727 - acc: 0.2655 - val_loss: 1.8415 - val_acc: 0.4433
Epoch 2/10
159/159 [==============================] - 5s 31ms/step - loss: 1.7249 - acc: 0.4525 - val_loss: 1.5354 - val_acc: 0.5348
Epoch 3/10
159/159 [==============================] - 5s 33ms/step - loss: 1.3752 - acc: 0.5756 - val_loss: 1.3480 - val_acc: 0.5947
Epoch 4/10
159/159 [==============================] - 5s 33ms/step - loss: 1.0849 - acc: 0.6792 - val_loss: 1.2214 - val_acc: 0.6327
Epoch 5/10
159/159 [==============================] - 5s 32ms/step - loss: 0.8369 - acc: 0.7595 - val_loss: 1.1418 - val_acc: 0.6496
Epoch 6/10
159/159 [==============================] - 5s 32ms/step - loss: 0.6489 - acc: 0.8165 - val_loss: 1.0922 - val_acc: 0.6640
Epoch 7/10
159/159 [==============================] - 4s 28ms/step - loss: 0.5120 - acc: 0.8573 - val_loss: 1.0685 - val_acc: 0.6711
Epoch 8/10
159/159 [==============================] - 5s 29ms/step -

In [17]:
myLSTM = Sequential()
myLSTM.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=47))
myLSTM.add(layers.Bidirectional(layers.LSTM(256, recurrent_dropout=0.3,return_sequences=True)))
myLSTM.add(GlobalMaxPooling1D())
myLSTM.add(Dense(64, activation='relu'))
myLSTM.add(Dense(15, activation='softmax'))
myLSTM.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])
myLSTM.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 47, 300)           24290700  
                                                                 
 bidirectional (Bidirectiona  (None, 47, 512)          1140736   
 l)                                                              
                                                                 
 global_max_pooling1d_1 (Glo  (None, 512)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 dense_2 (Dense)             (None, 15)                975       
                                                                 
Total params: 25,465,243
Trainable params: 25,465,243


In [18]:
history = myLSTM.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 


Epoch 1/10
159/159 [==============================] - 141s 856ms/step - loss: 1.7277 - acc: 0.4347 - val_loss: 1.3275 - val_acc: 0.5616 - lr: 0.0010
Epoch 2/10
159/159 [==============================] - 101s 637ms/step - loss: 0.9986 - acc: 0.6798 - val_loss: 0.9578 - val_acc: 0.6906 - lr: 0.0010
Epoch 3/10
159/159 [==============================] - 129s 815ms/step - loss: 0.5176 - acc: 0.8420 - val_loss: 0.9281 - val_acc: 0.7171 - lr: 0.0010
Epoch 4/10
159/159 [==============================] - 136s 855ms/step - loss: 0.2305 - acc: 0.9321 - val_loss: 1.0116 - val_acc: 0.7234 - lr: 0.0010
Epoch 5/10
159/159 [==============================] - 99s 621ms/step - loss: 0.1106 - acc: 0.9684 - val_loss: 1.1100 - val_acc: 0.7207 - lr: 0.0010
Epoch 6/10
159/159 [==============================] - 98s 618ms/step - loss: 0.0592 - acc: 0.9833 - val_loss: 1.1777 - val_acc: 0.7258 - lr: 0.0010


In [19]:
loss, accuracy = myLSTM.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myLSTM.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

635/635 [==============================] - 35s 56ms/step - loss: 0.0297 - acc: 0.9930
Training Accuracy: 0.9930
159/159 [==============================] - 9s 56ms/step - loss: 1.1777 - acc: 0.7258
Testing Accuracy:  0.7258


In [20]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, input_shape):
        self.W_in = self.add_weight(shape=(input_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
       
        self.b = self.add_weight(shape=(input_shape[1], 1),
                                        initializer='zeros',
                                        trainable=True,
                                        name='attention_bias')
        
        super(AttentionLayer, self).build(input_shape)
        
        
    def call(self, lstm_output):
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = lstm_output * at
        return attention_output
    
    
    def compute_output_shape(self, input_state):
        return (input_state[0], input_state[-1])
    
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

In [21]:
Input1 = Input(shape=(47,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.3, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
max_pool = MaxPooling1D(pool_size=5)(Attention)
Dropout1 = Dropout(0.3)(max_pool)
Flat = Flatten()(Dropout1)
Dense1 = Dense(512, activation='relu')(Flat)
Dropout2 = Dropout(0.3)(Dense1)
Dense2 = Dense(64, activation='relu')(Dropout2)
Dropout3 = Dropout(0.25)(Dense2)
outputs = Dense(15, activation='softmax')(Dropout3)

model = Model(inputs=Input1, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 47)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 47, 300)           24290700  
                                                                 
 bidirectional_1 (Bidirectio  (None, 47, 128)          186880    
 nal)                                                            
                                                                 
 attention_layer (AttentionL  (None, 47, 128)          175       
 ayer)                                                           
                                                                 
 max_pooling1d (MaxPooling1D  (None, 9, 128)           0         
 )                                                               
                                                             

In [31]:
history = model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
159/159 [==============================] - 8s 49ms/step - loss: 0.0793 - acc: 0.9773 - val_loss: 1.8397 - val_acc: 0.6679
Epoch 2/10
159/159 [==============================] - 8s 48ms/step - loss: 0.0679 - acc: 0.9798 - val_loss: 1.8145 - val_acc: 0.6657
Epoch 3/10
159/159 [==============================] - 8s 48ms/step - loss: 0.0570 - acc: 0.9828 - val_loss: 1.9724 - val_acc: 0.6738
Epoch 4/10
159/159 [==============================] - 8s 48ms/step - loss: 0.0581 - acc: 0.9823 - val_loss: 2.0649 - val_acc: 0.6541
Epoch 5/10
159/159 [==============================] - 8s 49ms/step - loss: 0.0495 - acc: 0.9851 - val_loss: 2.2601 - val_acc: 0.6589
Epoch 6/10
159/159 [==============================] - 8s 49ms/step - loss: 0.0439 - acc: 0.9878 - val_loss: 2.0688 - val_acc: 0.6650
Epoch 7/10
159/159 [==============================] - 8s 49ms/step - loss: 0.0352 - acc: 0.9896 - val_loss: 1.9983 - val_acc: 0.6758
Epoch 8/10
159/159 [==============================] - 8s 49ms/step - 

In [23]:
Input1 = Input(shape=(47,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.25, return_sequences=True))(Embedding_layer1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu')(LSTM1)
max_pooling1 = MaxPooling1D(pool_size=3)(Conv1)
Dropout2 = Dropout(0.25)(max_pooling1)
Flatten1 = Flatten()(Dropout2)
Dense1 = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(Flatten1)
Dense2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(Dense1)
output = Dense(15, activation='softmax')(Dense2)

myBiLSTM_CNN = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 47)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 47, 300)           24290700  
                                                                 
 bidirectional_2 (Bidirectio  (None, 47, 128)          186880    
 nal)                                                            
                                                                 
 conv1d_1 (Conv1D)           (None, 43, 64)            41024     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 14, 64)           0         
 1D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 64)            0   

In [24]:
history = myBiLSTM_CNN.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 
loss, accuracy = myBiLSTM_CNN.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myBiLSTM_CNN.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
159/159 [==============================] - 14s 60ms/step - loss: 3.3572 - acc: 0.3912 - val_loss: 1.7284 - val_acc: 0.5421
Epoch 2/10
159/159 [==============================] - 8s 52ms/step - loss: 1.3379 - acc: 0.6484 - val_loss: 1.2395 - val_acc: 0.6687
Epoch 3/10
159/159 [==============================] - 8s 52ms/step - loss: 0.7724 - acc: 0.8116 - val_loss: 1.1683 - val_acc: 0.6996
Epoch 4/10
159/159 [==============================] - 8s 52ms/step - loss: 0.4586 - acc: 0.9012 - val_loss: 1.2582 - val_acc: 0.6758
Epoch 5/10
159/159 [==============================] - 8s 52ms/step - loss: 0.3229 - acc: 0.9370 - val_loss: 1.2207 - val_acc: 0.7142
Epoch 6/10
159/159 [==============================] - 8s 52ms/step - loss: 0.2295 - acc: 0.9602 - val_loss: 1.3387 - val_acc: 0.7112
Epoch 7/10
159/159 [==============================] - 8s 52ms/step - loss: 0.1848 - acc: 0.9688 - val_loss: 1.4394 - val_acc: 0.6972
Epoch 8/10
159/159 [==============================] - 8s 52ms/step -

In [25]:
Input1 = Input(shape=(47,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)

LSTM1 = Bidirectional(LSTM(128, dropout=0.25, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu', padding='same')(Attention)
max_pool = GlobalMaxPooling1D()(Conv1)
Dropout2 = Dropout(0.25)(max_pool)
output = Dense(15, activation='softmax')(Dropout2)

myBiLSTM_CNN_Att = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN_Att.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN_Att.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 47)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 47, 300)           24290700  
                                                                 
 bidirectional_3 (Bidirectio  (None, 47, 256)          439296    
 nal)                                                            
                                                                 
 attention_layer_1 (Attentio  (None, 47, 256)          303       
 nLayer)                                                         
                                                                 
 conv1d_2 (Conv1D)           (None, 47, 64)            81984     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0   

In [30]:
history = myBiLSTM_CNN_Att.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,reduce_lr_callback]) 
loss, accuracy = myBiLSTM_CNN_Att.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myBiLSTM_CNN_Att.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
159/159 [==============================] - 10s 63ms/step - loss: 0.0471 - acc: 0.9869 - val_loss: 1.6086 - val_acc: 0.6976 - lr: 0.0010
Epoch 2/10
159/159 [==============================] - 10s 63ms/step - loss: 0.0401 - acc: 0.9880 - val_loss: 1.5631 - val_acc: 0.6947 - lr: 0.0010
Epoch 3/10
159/159 [==============================] - ETA: 0s - loss: 0.0322 - acc: 0.9912WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005.
159/159 [==============================] - 10s 63ms/step - loss: 0.0322 - acc: 0.9912 - val_loss: 1.5933 - val_acc: 0.6858 - lr: 0.0010
Epoch 4/10
159/159 [==============================] - 10s 63ms/step - loss: 0.0209 - acc: 0.9943 - val_loss: 1.6604 - val_acc: 0.7047 - lr: 5.0000e-04
Epoch 5/10
159/159 [==============================] - 10s 63ms/step - loss: 0.0157 - acc: 0.9959 - val_loss: 1.7252 - val_acc: 0.7031 - lr: 5.0000e-04
Epoch 6/10
159/159 [=========

In [27]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, attention_input):
        super(AttentionLayer, self).build(attention_input)
        lstm_shape, cnn_shape = attention_input
        # Attention Weights for LSTM
        self.W_in = self.add_weight(shape=(lstm_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
        
        # Attention Weights for CNN
        self.W_context = self.add_weight(shape=(cnn_shape[-1], 1),
                                         initializer='glorot_normal',
                                         trainable=True,
                                         name='context_attention_weights')
                                      
        # Attention Bias
        self.b = self.add_weight(shape=(lstm_shape[1], 1),
                                 initializer='glorot_normal',
                                 trainable=True,
                                 name='attention_bias')
        
        
        
    def call(self, attention_input):
        lstm_output, cnn_output = attention_input
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) + 
                              K.dot(cnn_output, self.W_context) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = at * lstm_output
        attention_output = K.sum(attention_output, axis=1)
        return attention_output
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

In [28]:
Input1 = Input(shape=(47,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)

# Input 1(BiLSTM):
Lstm = Bidirectional(LSTM(128, dropout=0.3, return_sequences=True))(Embedding_layer1)

# Input 2 (CNN):
Conv1 = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(Embedding_layer1)
Dropout1 = Dropout(0.3)(Conv1)
Conv2 = Conv1D(filters=256, kernel_size=4, activation='relu', padding='same')(Embedding_layer1)
Dropout2 = Dropout(0.3)(Conv2)
Conv3 = Conv1D(filters=256, kernel_size=5, activation='relu', padding='same')(Embedding_layer1)
Dropout3 = Dropout(0.3)(Conv3)
merged = concatenate([Dropout1, Dropout2, Dropout3], axis=1)
max_pool = MaxPooling1D(pool_size=3)(merged)

attention = AttentionLayer()([Lstm, max_pool])
Dense1 = Dense(64, activation='relu')(attention)
Dropout_dense = Dropout(0.25)(Dense1)
outputs = Dense(15, activation='softmax')(Dropout_dense)

classifier = Model(inputs=Input1, outputs=outputs)
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
classifier.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 47)]         0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 47, 300)      24290700    ['input_4[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 47, 256)      230656      ['embedding_5[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 47, 256)      307456      ['embedding_5[0][0]']            
                                                                                            

In [29]:
history = classifier.fit(X_train, y_train, epochs=16, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,reduce_lr_callback]) 
loss, accuracy = classifier.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = classifier.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/16
159/159 [==============================] - 24s 107ms/step - loss: 1.9175 - acc: 0.3662 - val_loss: 1.4727 - val_acc: 0.5266 - lr: 0.0010
Epoch 2/16
159/159 [==============================] - 14s 89ms/step - loss: 1.2114 - acc: 0.6175 - val_loss: 1.0552 - val_acc: 0.6671 - lr: 0.0010
Epoch 3/16
159/159 [==============================] - 14s 90ms/step - loss: 0.7381 - acc: 0.7739 - val_loss: 0.9758 - val_acc: 0.6961 - lr: 0.0010
Epoch 4/16
159/159 [==============================] - 14s 90ms/step - loss: 0.4225 - acc: 0.8751 - val_loss: 0.9289 - val_acc: 0.7329 - lr: 0.0010
Epoch 5/16
159/159 [==============================] - 14s 90ms/step - loss: 0.2416 - acc: 0.9313 - val_loss: 1.0043 - val_acc: 0.7309 - lr: 0.0010
Epoch 6/16
159/159 [==============================] - ETA: 0s - loss: 0.1519 - acc: 0.9566WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005.
159/159 [================